# Aula 03 - SQL para Analytics: Join and Having in SQL

## Introdução aos Joins em SQL

Joins em SQL são fundamentais para combinar registros de duas ou mais tabelas em um banco de dados com base em uma condição comum, geralmente uma chave estrangeira. Essa técnica permite que dados relacionados, que são armazenados em tabelas separadas, sejam consultados juntos de forma eficiente e coerente. 

Os joins são essenciais para consultar dados complexos e para aplicações em que a normalização do banco de dados resulta em distribuição de informações por diversas tabelas.

Existem vários tipos de joins, cada um com seu uso específico dependendo das necessidades da consulta:

1. **Inner Join**: Retorna registros que têm correspondência em ambas as tabelas.
2. **Left Join (ou Left Outer Join)**: Retorna todos os registros da tabela esquerda e os registros correspondentes da tabela direita. Se não houver correspondência, os resultados da tabela direita terão valores `NULL`.
3. **Right Join (ou Right Outer Join)**: Retorna todos os registros da tabela direita e os registros correspondentes da tabela esquerda. Se não houver correspondência, os resultados da tabela esquerda terão valores `NULL`.
4. **Full Join (ou Full Outer Join)**: Retorna registros quando há uma correspondência em uma das tabelas. Se não houver correspondência, ainda assim, o resultado aparecerá com `NULL` nos campos da tabela sem correspondência.

# Primeiro vamos criar uma conexão com o banco de dados:

In [1]:
import psycopg
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Conectar ao banco de dados PostgreSQL
conn = psycopg.connect("dbname=postgres user=postgres password=password host=172.25.224.1 port=5432")
cursor = conn.cursor()

### 1. Criar um relatório para todos os pedidos de 1996 e seus clientes

**Inner Join**

**Uso**: Utilizado quando você precisa de registros que têm correspondência exata em ambas as tabelas. 

**Exemplo Prático**: Se quisermos encontrar todos os pedidos de 1996 e os detalhes dos clientes que fizeram esses pedidos, usamos um Inner Join. Isso garante que só obteremos os pedidos que possuem um cliente correspondente e que foram feitos em 1996.

In [13]:
query = '''
-- 1. Cria um relatório para todos os pedidos de 1996 e seus clientes (152 linhas)
SELECT *
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
WHERE DATE_PART('YEAR', o.order_date) = 1996;
'''
pd.read_sql(query, conn)

,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,...,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,10308,ANATR,7,1996-09-18,1996-10-16,1996-09-24,3,1.61,Ana Trujillo Emparedados y helados,Avda. de la Constitución 2222,...,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
1,10365,ANTON,3,1996-11-27,1996-12-25,1996-12-02,2,22.00,Antonio Moreno Taquería,Mataderos 2312,...,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
2,10383,AROUT,8,1996-12-16,1997-01-13,1996-12-18,3,34.24,Around the Horn,Brook Farm Stratford St. Mary,...,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
3,10355,AROUT,6,1996-11-15,1996-12-13,1996-11-20,1,41.95,Around the Horn,Brook Farm Stratford St. Mary,...,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,10384,BERGS,3,1996-12-16,1997-01-13,1996-12-20,3,168.64,Berglunds snabbköp,Berguvsvägen 8,...,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,10266,WARTH,3,1996-07-26,1996-09-06,1996-07-31,3,25.73,Wartian Herkku,Torikatu 38,...,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Torikatu 38,Oulu,None,90110,Finland,981-443655,981-443655
148,10256,WELLI,3,1996-07-15,1996-08-12,1996-07-17,2,13.97,Wellington Importadora,"Rua do Mercado, 12",...,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil,(14) 555-8122,None
149,10344,WHITC,4,1996-11-01,1996-11-29,1996-11-05,2,23.29,White Clover Markets,1029 - 12th Ave. S.,...,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
150,10269,WHITC,5,1996-07-31,1996-08-14,1996-08-09,1,4.56,White Clover Markets,1029 - 12th Ave. S.,...,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115


**Vantagens:**

* **`DATE_PART` para extrair o ano:**  Em vez de usar `BETWEEN` para definir um intervalo de datas,  `DATE_PART('YEAR', o.order_date)` extrai apenas o ano da coluna `order_date`, tornando a consulta mais concisa.
* **Comparação direta:** O comando `WHERE DATE_PART('YEAR', o.order_date) = 1996` é mais simples e direto do que o `BETWEEN` usado no meu exemplo anterior.

**Observações:**

* `DATE_PART` é uma função específica do PostgreSQL. Se você estiver usando outro SGBD, como MySQL ou SQL Server, precisará usar uma função equivalente para extrair o ano da data. 
* `SELECT *` irá retornar todas as colunas das duas tabelas. Se você quiser retornar apenas colunas específicas, liste-as na cláusula `SELECT` como no meu primeiro exemplo.

### 2. Criar um relatório que mostra o número de funcionários e clientes de cada cidade que tem funcionários

**Left Join**

**Uso**: Usado quando você quer todos os registros da primeira (esquerda) tabela, com os correspondentes da segunda (direita) tabela. Se não houver correspondência, a segunda tabela terá campos `NULL`. 

**Exemplo Prático**: Se precisarmos listar todas as cidades onde temos funcionários, e também queremos saber quantos clientes temos nessas cidades, mesmo que não haja clientes, usamos um Left Join.


In [6]:
query = '''
-- Cria um relatório que mostra o número de funcionários e clientes de cada cidade que tem funcionários (5 linhas)
SELECT e.city AS cidade, 
       COUNT(DISTINCT e.employee_id) AS numero_de_funcionarios, 
       COUNT(DISTINCT c.customer_id) AS numero_de_clientes
FROM employees e 
LEFT JOIN customers c ON e.city = c.city
GROUP BY e.city
ORDER BY cidade;
'''
pd.read_sql(query, conn)

,cidade,numero_de_funcionarios,numero_de_clientes
0,Kirkland,1,1
1,London,4,6
2,Redmond,1,0
3,Seattle,2,1
4,Tacoma,1,0


### Descrição da Tabela

* **cidade**: O nome da cidade onde os funcionários e clientes estão localizados.
* **numero_de_funcionarios**: Contagem dos funcionários distintos nessa cidade. Este número vem diretamente da tabela `employees`.
* **numero_de_clientes**: Contagem dos clientes distintos que têm a mesma cidade que os funcionários. Se não houver clientes em uma cidade onde há funcionários, o número será 0.

### Explicação Detalhada

* **Kirkland**: Tem um equilíbrio entre o número de funcionários e clientes, com ambos os valores sendo 1. Isso indica uma correspondência direta entre locais de funcionários e clientes.
* **London**: Apresenta uma maior concentração tanto de funcionários quanto de clientes, com mais clientes (6) do que funcionários (4), indicando uma forte presença de ambos na cidade.
* **Redmond**: Tem 1 funcionário, mas nenhum cliente registrado nesta cidade, sugerindo que, embora a empresa tenha presença laboral aqui, não há clientes registrados.
* **Seattle**: Tem 2 funcionários e apenas 1 cliente, mostrando uma presença menor de clientes em relação aos funcionários.
* **Tacoma**: Similar a Redmond, tem funcionários (1) mas nenhum cliente, o que pode indicar uma área onde a empresa opera, mas ainda não estabeleceu uma base de clientes.

Essa análise é particularmente útil para entender como os recursos humanos da empresa (funcionários) estão distribuídos em relação à sua base de clientes em diferentes locais. Isso pode ajudar a identificar cidades onde a empresa pode precisar intensificar esforços de aquisição de clientes ou avaliar a eficácia de suas operações e estratégias de mercado locais.

### 3. Criar um relatório que mostra o número de funcionários e clientes de cada cidade que tem clientes

**Right Join**

**Uso**: É o inverso do Left Join e é menos comum. Usado quando queremos todos os registros da segunda (direita) tabela e os correspondentes da primeira (esquerda) tabela. 

**Exemplo Prático**: Para listar todas as cidades onde temos clientes, e também contar quantos funcionários temos nessas cidades, usamos um Right Join.

In [8]:
query = ''' 
-- Cria um relatório que mostra o número de funcionários e clientes de cada cidade que tem clientes (69 linhas)
SELECT c.city AS cidade, 
       COUNT(DISTINCT c.customer_id) AS numero_de_clientes, 
       COUNT(DISTINCT e.employee_id) AS numero_de_funcionarios
FROM employees e 
RIGHT JOIN customers c ON e.city = c.city
GROUP BY c.city
ORDER BY cidade;
'''
pd.read_sql(query, conn)

,cidade,numero_de_clientes,numero_de_funcionarios
0,Aachen,1,0
1,Albuquerque,1,0
2,Anchorage,1,0
3,Århus,1,0
4,Barcelona,1,0
...,...,...,...
64,Tsawassen,1,0
65,Vancouver,1,0
66,Versailles,1,0
67,Walla Walla,1,0


### Diferenças Principais do `RIGHT JOIN`

1. **Foco na Tabela à Direita**: Ao contrário do `LEFT JOIN` que foca na tabela à esquerda, o `RIGHT JOIN` garante que todos os registros da tabela à direita (neste caso, `customers`) estejam presentes no resultado. Se não houver correspondência na tabela à esquerda (`employees`), as colunas relacionadas desta tabela aparecerão como `NULL`.
    
2. **Exibição de Dados Não Correspondentes**: Como mostrado, o `RIGHT JOIN` pode exibir linhas onde não há correspondência na tabela à esquerda, o que é útil para identificar dados que estão apenas na tabela à direita. No contexto de um negócio, isso pode destacar áreas (ou dados) que requerem atenção, como clientes em locais onde a empresa não tem funcionários representados.
    
3. **Utilização Estratégica para Análise de Dados**: O `RIGHT JOIN` é menos comum que o `LEFT JOIN` porque muitas vezes as tabelas são organizadas de modo que a tabela mais importante (ou abrangente) seja colocada à esquerda da consulta. No entanto, o `RIGHT JOIN` é útil quando a tabela à direita é prioritária e queremos garantir que todos os seus registros sejam analisados.

### 4. Criar um relatório que mostra o número de funcionários e clientes de cada cidade

* **Análise Completa de Dados**: O `FULL JOIN` é útil quando você precisa de uma visão completa dos dados em duas tabelas relacionadas, especialmente para identificar onde os dados estão faltando em uma ou ambas as tabelas.
* **Relatórios Abrangentes**: Permite criar relatórios que mostram todas as possíveis relações entre duas tabelas, incluindo onde as relações não existem.
* **Análise de Lacunas de Dados**: Ajuda a identificar lacunas nos dados de ambas as tabelas simultaneamente, facilitando análises de cobertura e consistência entre conjuntos de dados.

**Full Join**

**Uso**: Utilizado quando queremos a união de Left Join e Right Join, mostrando todos os registros de ambas as tabelas, e preenchendo com `NULL` onde não há correspondência. 

**Exemplo Prático**: Para listar todas as cidades onde temos clientes ou funcionários, e contar ambos em cada cidade, usamos um Full Join.

In [9]:
query = ''' 
-- Cria um relatório que mostra o número de funcionários e clientes de cada cidade (71 linhas)
SELECT
	COALESCE(e.city, c.city) AS cidade,
	COUNT(DISTINCT e.employee_id) AS numero_de_funcionarios,
	COUNT(DISTINCT c.customer_id) AS numero_de_clientes
FROM employees e 
FULL JOIN customers c ON e.city = c.city
GROUP BY e.city, c.city
ORDER BY cidade;
'''
pd.read_sql(query, conn)

,cidade,numero_de_funcionarios,numero_de_clientes
0,Aachen,0,1
1,Albuquerque,0,1
2,Anchorage,0,1
3,Århus,0,1
4,Barcelona,0,1
...,...,...,...
66,Tsawassen,0,1
67,Vancouver,0,1
68,Versailles,0,1
69,Walla Walla,0,1


Esta consulta retorna uma lista de todas as cidades conhecidas por ambas as tabelas, junto com a contagem de funcionários e clientes em cada cidade. Aqui estão alguns cenários possíveis no resultado:

### Análise do Resultado

O resultado do `FULL JOIN` mostra:

* A maioria das cidades listadas tem clientes, mas não funcionários (indicado por "0" no número de funcionários).
* Em algumas cidades, como "Kirkland", "Redmond", "Seattle", e "Tacoma", há funcionários e/ou clientes, mostrando correspondência direta entre as tabelas.
* Notavelmente, em cidades como "London" e "Madrid", o número de clientes é significativamente maior do que o de funcionários, o que pode indicar centros de alta atividade de clientes sem uma proporção correspondente de suporte de funcionários.

### Observações Importantes

1. **Cidades com apenas Clientes**: A maioria das cidades no resultado possui clientes, mas não funcionários. Isso pode sugerir que a empresa tem uma ampla base de clientes geograficamente, mas uma distribuição mais limitada de sua força de trabalho.
    
2. **Cidades com Funcionários e sem Clientes**: Cidades como "Redmond" e "Tacoma" têm funcionários, mas nenhuma contagem de clientes listada, indicando que há operações da empresa sem correspondente atividade de clientes registrada nesses locais.
    
3. **Concentrações de Clientes e Funcionários**: Em cidades como "London", "Seattle", e "Sao Paulo", há uma concentração significativa de clientes e alguma presença de funcionários, sugerindo centros operacionais ou mercados importantes para a empresa.
    
4. **Ausência de Dados em Algumas Cidades**: Algumas cidades têm zero funcionários e clientes, indicando que pode haver um erro de dados, cidades listadas incorretamente, ou simplesmente que não há atividade de funcionários ou clientes registrados nesses locais.
    

### Implicações Estratégicas

A partir desses dados, a empresa poderia considerar várias ações estratégicas:

* **Expansão de Funcionários**: Investir em recursos humanos nas cidades com altos números de clientes, mas baixa presença de funcionários, para melhorar o suporte e a satisfação do cliente.
    
* **Análise de Mercado**: Realizar uma análise mais aprofundada sobre por que certas cidades têm alta atividade de clientes e ajustar as estratégias de marketing e vendas conforme necessário.
    
* **Revisão de Dados**: Verificar a precisão dos dados para entender melhor as discrepâncias ou ausências nas contagens de funcionários e clientes.
    
Este exemplo realça o valor de usar `FULL JOIN` para obter uma visão completa da relação entre duas variáveis críticas (funcionários e clientes) e como essa informação pode ser usada para insights estratégicos.

## Having

### 1. Criar um relatório que mostra a quantidade total de produtos (da tabela order_details)

In [10]:
query = ''' 
-- Cria um relatório que mostra a quantidade total de produtos encomendados.
-- Mostra apenas registros para produtos para os quais a quantidade encomendada é menor que 200 (5 linhas)
SELECT o.product_id, p.product_name, SUM(o.quantity) AS quantidade_total
FROM order_details o
JOIN products p ON p.product_id = o.product_id
GROUP BY o.product_id, p.product_name
HAVING SUM(o.quantity) < 200
ORDER BY quantidade_total DESC;
'''
pd.read_sql(query, conn)


,product_id,product_name,quantidade_total
0,67,Laughing Lumberjack Lager,184
1,48,Chocolade,138
2,37,Gravad lax,125
3,15,Genen Shouyu,122
4,9,Mishi Kobe Niku,95


### 2. Criar um relatório que mostra o total de pedidos por cliente desde 31 de dezembro de 1996

In [12]:
query = ''' 
-- Cria um relatório que mostra o total de pedidos por cliente desde 31 de dezembro de 1996.
SELECT customer_id, COUNT(order_id) AS total_de_pedidos
FROM orders
WHERE order_date > '1996-12-31'
GROUP BY customer_id
HAVING COUNT(order_id) > 15
ORDER BY total_de_pedidos;
'''
pd.read_sql(query, conn)

,customer_id,total_de_pedidos
0,FOLKO,16
1,HILAA,16
2,QUICK,22
3,ERNSH,24
4,SAVEA,28


### Explicação das Consultas Convertidas

**Consulta 1:**

* **Seleção e Junção**: A consulta seleciona o `product_id` e `product_name` da tabela `products` e junta com a tabela `order_details` pelo `product_id`.
* **Agrupamento e Filtragem**: Os dados são agrupados por `product_id` e `product_name`, e a função agregada `SUM(o.quantity)` calcula a quantidade total de cada produto encomendado. A cláusula `HAVING` é usada para filtrar produtos cuja quantidade total encomendada é menor que 200.

**Consulta 2:**

* **Filtragem de Data**: A consulta filtra os pedidos realizados após 31 de dezembro de 1996.
* **Agrupamento e Contagem**: Agrupa os pedidos pelo `customer_id` e conta o número de pedidos feitos por cada cliente usando `COUNT(order_id)`.
* **Filtragem de Resultados**: Utiliza a cláusula `HAVING` para incluir apenas os clientes que fizeram mais de 15 pedidos desde a data especificada.